# Ingestion Service Testing (Stage 3)

## Objective
Validate `IngestionService` behavior in isolation, without running Metaflow.

## Prerequisites
1. Run from repository root.
2. Have at least one PDF in `data/`.
3. Python environment with project dependencies installed.


In [ ]:
from pathlib import Path

from notebooks._utils import data_dir, list_pdfs, print_checkpoint, project_root
from src.ingest.service import IngestionService

ROOT = project_root()
DATA_DIR = data_dir()
PDFS = list_pdfs(DATA_DIR)

print("ROOT:", ROOT)
print("DATA_DIR:", DATA_DIR)
print(f"Found {len(PDFS)} PDFs")
assert PDFS, "No PDF files found in data/"
print_checkpoint("setup complete")


## 1) Discovery behavior

Checks:
1. files are discovered recursively
2. result is deterministic (sorted)


In [ ]:
service = IngestionService()
files = service.discover_pdf_files(DATA_DIR)

assert files == sorted(files), "discover_pdf_files must return sorted results"
assert all(p.suffix.lower() == ".pdf" for p in files), "discovery should include only PDFs"

print("First 5 discovered files:")
for p in files[:5]:
    print("-", p.name)
print_checkpoint("discovery assertions passed")


## 2) Parser invocation through service

Parse a single sample PDF via service boundary.


In [ ]:
sample = files[0]
parsed = service.parse_pdf(sample)

assert parsed.sections, "Expected non-empty parsed sections"
assert parsed.parser_version, "Expected parser version"

print("sample:", sample.name)
print("language:", parsed.language)
print("sections:", list(parsed.sections.keys()))
print("links:", parsed.links[:5])
print_checkpoint("service parse assertions passed")


## 3) Deterministic helper checks

Validate deterministic identity helper behavior.


In [ ]:
sample = files[0]
external_id_1 = service._build_candidate_external_id(sample)
external_id_2 = service._build_candidate_external_id(sample)

assert external_id_1 == external_id_2, "external_id helper should be deterministic"
assert external_id_1.startswith("resume_file:"), "external_id prefix changed unexpectedly"

name = service._infer_candidate_name(sample)
assert isinstance(name, str) and name.strip(), "inferred candidate name should be non-empty"

print("external_id:", external_id_1)
print("inferred_name:", name)
print_checkpoint("helper assertions passed")


## 4) Dry-run ingestion simulation notes

This notebook intentionally does not run `ingest_pdf` against DB by default.

If you want full persistence checks, use `repositories_smoke_testing.ipynb` and/or run Metaflow flow.


## Summary (fill after run)

- Discovery assertions: pass/fail
- Parse-through-service assertions: pass/fail
- Deterministic helper assertions: pass/fail


## Next actions

1. If parse results look wrong, debug in `parsers_testing.ipynb` first.
2. If service helpers break, add/update unit tests in `tests/test_ingest_service.py`.
